In [2]:
#!/usr/bin/env python3
"""
═══════════════════════════════════════════════════════════════════════════════
  RMR Four-Force Simulation Engine v6 — Relational Potential
═══════════════════════════════════════════════════════════════════════════════

  Companion code for:
    "Emergent Four-Force Dynamics from a Discrete 137-Bit Registry:
     Gravity, Electromagnetism, Strong, and Weak Interactions
     via Causal Integer Lattice Simulation"
    J. R. Merwin (2026)

═══════════════════════════════════════════════════════════════════════════════
  OVERVIEW
═══════════════════════════════════════════════════════════════════════════════

  A causal integer lattice simulation in which four qualitatively distinct
  force behaviors emerge from a SINGLE overflow-fission rule governed by
  one thermodynamic equation:

                            E  =  q · Φ

  where q is the mass's charge in the overflowing sector and Φ is the local
  field saturation at a candidate neighbor.  The mass relocates to whichever
  neighbor minimizes E, producing:

    • Gravity  — unsigned depletion gradient (q=1, Φ≥0 → seeks lowest Φ)
    • EM       — signed annihilation/resistance (q=±1, Φ signed → attracts
                 opposite, repels like)
    • Color    — high-capacity color confinement (same sign logic, 81-bit
                 sector → aggressive short-range saturation)
    • Weak     — adjacency-triggered flavor change (P = n_adj / 137)

  No force laws, coupling constants, or interaction-specific logic appear
  anywhere in the engine.  The diversity of forces emerges entirely from
  the 137-bit registry partition (16 + 40 + 81) and the sign structure
  of the charges.

═══════════════════════════════════════════════════════════════════════════════
  DESIGN PRINCIPLES
═══════════════════════════════════════════════════════════════════════════════

  1. INTEGER ONLY — All field values are int32.  No floating-point arithmetic
     touches the physics.  This is not a performance choice; it is required
     for the depletion mechanism to function (see paper §4.2).

  2. CAUSAL PROPAGATION — Nodes update in random sequential order each tick.
     No simultaneous updates.  This eliminates the hidden global clock that
     synchronous lattice updates smuggle in (see paper §4.2).

  3. SINGLE RULE — Overflow → fission via E = qΦ.  Gravity, EM, strong, and
     weak forces are NOT coded separately.  They emerge from the sector
     partition and charge signs.

  4. TIMING ASYMMETRY — Mass ticks every 5 steps, vacuum every 4 steps.
     The 5/4 ratio creates the depletion gradient that drives gravity.

═══════════════════════════════════════════════════════════════════════════════
  REGISTRY ARCHITECTURE
═══════════════════════════════════════════════════════════════════════════════

        Sector      Capacity    Signedness    Radiation Rate
        ─────────   ────────    ──────────    ──────────────
        Gravity        16       unsigned         1 / mass tick
        EM             40       signed (±)       3 / mass tick
        Color          81       signed (±)       5 / mass tick
        ─────────   ────────
        TOTAL         137       = 1/α (fine structure constant)

═══════════════════════════════════════════════════════════════════════════════
  USAGE
═══════════════════════════════════════════════════════════════════════════════

  Quick start (two neutral masses, gravity only):

      from rmr_engine_v6 import Sim

      sim = Sim(S=35, seed=42)
      sim.add(15, 17, 17, q_em=0, q_col=0)
      sim.add(19, 17, 17, q_em=0, q_col=0)

      for tick in range(3000):
          stats = sim.step()

      print(f"Final separation: {sim.mean_separation():.1f}")

  Full validation suite (reproduces Table 1 from the paper):

      python rmr_engine_v6.py

  Dependencies:  numpy, numba, scipy

═══════════════════════════════════════════════════════════════════════════════
  VERSION HISTORY (abbreviated; see paper §4.2 for full discussion)
═══════════════════════════════════════════════════════════════════════════════

  v1–v2  Swap/reflect logic.  Gravity confirmed; EM annihilation confirmed.
  v3     Additive regeneration (mass as net sink).  Weak force improved.
  v4     Total pressure gradient.  FAILURE: gravitational masking — gravity
         dominated all sectors, producing anomalous like-charge annihilation.
         Lesson: the 137-bit registry is partitioned, not pooled.
  v5     Sector-specific |Φ| gradient.  FAILURE: absolute value catastrophe —
         opposite charges repelled because |Φ| converted attractive sinks
         into apparent mountains.
         Lesson: force is relational, not absolute.
  v6     Relational potential E = qΦ.  All four forces confirmed.  This is
         the version described in the paper and released here.

═══════════════════════════════════════════════════════════════════════════════
"""

import numpy as np
from numba import njit, uint32
import time
from scipy import stats as sp_stats


# ═══════════════════════════════════════════════════════════════════════════
#   REGISTRY CONSTANTS
# ═══════════════════════════════════════════════════════════════════════════

GRAV_CAP    = 16          # Gravitational sector capacity (unsigned)
EM_CAP      = 40          # Electromagnetic sector capacity (signed ±)
COLOR_CAP   = 81          # Color sector capacity (signed ±)
TOTAL_CAP   = 137         # = GRAV_CAP + EM_CAP + COLOR_CAP = 1/α

VAC_PERIOD  = 4           # Vacuum processing period (ticks)
MASS_PERIOD = 5           # Mass processing period (ticks)
                          # Ratio = 5/4 → timing asymmetry

GRAV_RAD    = 1           # Gravitational quanta per mass tick  ⌈16/16⌉
EM_RAD      = 3           # EM quanta per mass tick             ⌈40/16⌉
COLOR_RAD   = 5           # Color quanta per mass tick          ⌈81/16⌉

# Six nearest neighbors on a 3D cubic lattice
NBR = np.array([
    [+1, 0, 0], [-1, 0, 0],
    [ 0,+1, 0], [ 0,-1, 0],
    [ 0, 0,+1], [ 0, 0,-1],
], dtype=np.int32)


# ═══════════════════════════════════════════════════════════════════════════
#   JIT-COMPILED CORE (numba)
# ═══════════════════════════════════════════════════════════════════════════

@njit(cache=True)
def _xor(s):
    """Xorshift32 PRNG.  Fast, deterministic, no floating-point."""
    s = uint32(s)
    s ^= uint32(s << 13)
    s ^= uint32(s >> 17)
    s ^= uint32(s << 5)
    return s


@njit(cache=True)
def _tdist(a, b, S):
    """Toroidal distance along one axis."""
    d = abs(a - b)
    return d if d <= S // 2 else S - d


@njit(cache=True)
def _eucl(x1, y1, z1, x2, y2, z2, S):
    """Euclidean distance on a 3D torus."""
    return (_tdist(x1, x2, S)**2 +
            _tdist(y1, y2, S)**2 +
            _tdist(z1, z2, S)**2)**0.5


@njit(cache=True)
def _do_fission(grav, em, color, mass_mask, charge, col_charge,
                mpx, mpy, mpz, nm, mx, my, mz, sx, sy, sz, S, ftype, rng):
    """
    CORE MECHANISM: Overflow-triggered fission via E = qΦ.

    When a mass at (mx,my,mz) overflows in sector `ftype`, it evaluates the
    relational potential E = q × Φ at all six neighbors and relocates to the
    cell that minimizes E.

    Parameters
    ----------
    grav, em, color : int32 3D arrays
        Field lattices for each sector.
    mass_mask : bool 3D array
        True where a mass node exists.
    charge, col_charge : int8 3D arrays
        EM and color charges at each site.
    mpx, mpy, mpz : int32 1D arrays
        Mass position tracking arrays.
    nm : int
        Number of masses.
    mx, my, mz : int
        Position of the overflowing mass.
    sx, sy, sz : int
        Position of the sender (source of the overflow quantum).
    S : int
        Lattice size.
    ftype : int
        Overflowing sector: 0=gravity, 1=EM, 2=color.
    rng : uint32
        PRNG state.

    Returns
    -------
    rng : uint32
        Updated PRNG state.
    displacement : float
        -1.0 if mass moved toward other masses (attractive),
        +1.0 if away (repulsive), 0.0 if no net change.
    """
    q  = charge[mx, my, mz]
    cq = col_charge[mx, my, mz]

    # ── Evaluate E = qΦ at all six neighbors ──
    best_e = 999999
    best_nx, best_ny, best_nz = mx, my, mz
    n_tied = 0

    for ni in range(6):
        nx = (mx + NBR[ni, 0]) % S
        ny = (my + NBR[ni, 1]) % S
        nz = (mz + NBR[ni, 2]) % S

        # Cannot fission into an occupied site
        if mass_mask[nx, ny, nz]:
            continue

        # Relational potential: E = q × Φ
        if ftype == 0:                          # Gravity (unsigned)
            e = grav[nx, ny, nz]                #   q = 1 implicitly
        elif ftype == 1:                        # Electromagnetism
            e = q * em[nx, ny, nz]              #   E = q_em × Φ_em
        else:                                   # Color
            e = cq * color[nx, ny, nz]          #   E = c_q × Φ_color

        if e < best_e:
            best_e = e
            best_nx, best_ny, best_nz = nx, ny, nz
            n_tied = 1
        elif e == best_e:
            n_tied += 1
            # Tiebreaker: prefer sender direction (causal bias)
            if nx == sx and ny == sy and nz == sz:
                best_nx, best_ny, best_nz = nx, ny, nz
            elif n_tied > 1:
                rng = _xor(rng)
                if rng % n_tied == 0:
                    best_nx, best_ny, best_nz = nx, ny, nz

    tx, ty, tz = best_nx, best_ny, best_nz

    # ── Compute net displacement relative to all other masses ──
    dist_change = 0.0
    for i in range(nm):
        if mpx[i] >= 0 and not (mpx[i] == mx and mpy[i] == my and mpz[i] == mz):
            db = _eucl(mx, my, mz, mpx[i], mpy[i], mpz[i], S)
            da = _eucl(tx, ty, tz, mpx[i], mpy[i], mpz[i], S)
            dist_change += (da - db)

    # ── Execute the move ──
    if not (tx == mx and ty == my and tz == mz):
        mass_mask[tx, ty, tz] = True
        charge[tx, ty, tz]     = q
        col_charge[tx, ty, tz] = cq
        grav[tx, ty, tz]       = GRAV_CAP
        em[tx, ty, tz]         = q  * EM_CAP
        color[tx, ty, tz]      = cq * COLOR_CAP

        mass_mask[mx, my, mz]  = False
        charge[mx, my, mz]     = 0
        col_charge[mx, my, mz] = 0
        grav[mx, my, mz]       = 0
        em[mx, my, mz]         = 0
        color[mx, my, mz]      = 0

        for i in range(nm):
            if mpx[i] == mx and mpy[i] == my and mpz[i] == mz:
                mpx[i] = tx; mpy[i] = ty; mpz[i] = tz
                break

    if dist_change < -0.1:   return rng, -1.0
    elif dist_change > 0.1:  return rng,  1.0
    return rng, 0.0


@njit(cache=True)
def _try_weak_decay(grav, em, color, mass_mask, charge, col_charge,
                    mpx, mpy, mpz, nm, x, y, z, S, rng):
    """
    Weak decay: adjacency-triggered flavor change.

    When a mass at (x,y,z) has n_adj adjacent mass neighbors, decay occurs
    with probability P = n_adj / 137.  The parent flips EM charge and a
    daughter particle is emitted into a random vacant neighbor, carrying the
    compensating charge.

    This is NOT a field-mediated force — it is a topological failure mode
    triggered when adjacent registries cannot maintain independent 137-bit
    partitions in overlapping space.
    """
    # Count adjacent masses
    n_adj = 0
    for ni in range(6):
        nx = (x + NBR[ni, 0]) % S
        ny = (y + NBR[ni, 1]) % S
        nz = (z + NBR[ni, 2]) % S
        if mass_mask[nx, ny, nz]:
            n_adj += 1

    if n_adj == 0:
        return rng, False

    # Decay probability: n_adj / 137
    rng = _xor(rng)
    if (rng % TOTAL_CAP) >= uint32(n_adj):
        return rng, False

    # Find a vacant neighbor for the daughter
    found = False
    tx, ty, tz = 0, 0, 0
    for attempt in range(6):
        rng = _xor(rng)
        ni = rng % 6
        cx = (x + NBR[ni, 0]) % S
        cy = (y + NBR[ni, 1]) % S
        cz = (z + NBR[ni, 2]) % S
        if not mass_mask[cx, cy, cz]:
            tx, ty, tz = cx, cy, cz
            found = True
            break

    if not found:
        return rng, False

    # Execute decay: parent flips charge, daughter carries complement
    q_old  = charge[x, y, z]
    cq_old = col_charge[x, y, z]
    q_new = -q_old if q_old != 0 else 1
    q_daughter = -q_new
    cq_daughter = -cq_old if cq_old != 0 else 0

    charge[x, y, z] = q_new
    em[x, y, z]     = q_new * EM_CAP

    mass_mask[tx, ty, tz]  = True
    charge[tx, ty, tz]     = q_daughter
    col_charge[tx, ty, tz] = cq_daughter
    grav[tx, ty, tz]       = GRAV_CAP
    em[tx, ty, tz]         = q_daughter  * EM_CAP
    color[tx, ty, tz]      = cq_daughter * COLOR_CAP

    for i in range(len(mpx)):
        if mpx[i] < 0:
            mpx[i] = tx; mpy[i] = ty; mpz[i] = tz
            break

    return rng, True


@njit(cache=True)
def _tick(grav, em, color, mass_mask, charge, col_charge,
          mpx, mpy, mpz, nm, S, t, rng):
    """
    Execute one tick of the simulation.

    Processing order is randomized each tick (causal propagation — no global
    clock).  Mass nodes radiate on MASS_PERIOD ticks; vacuum nodes diffuse on
    VAC_PERIOD ticks.  Overflow triggers fission via E = qΦ.

    Returns
    -------
    rng : uint32
        Updated PRNG state.
    stats : int32[9]
        [grav_toward, grav_away, em_toward, em_away, color_toward, color_away,
         em_annihilation, color_annihilation, weak_decay]
    """
    stats = np.zeros(9, dtype=np.int32)
    N = S * S * S

    # ── Randomize processing order (Fisher-Yates shuffle) ──
    order = np.arange(N, dtype=np.int32)
    for i in range(N - 1, 0, -1):
        rng = _xor(rng)
        j = rng % (i + 1)
        order[i], order[j] = order[j], order[i]

    for oi in range(N):
        idx = order[oi]
        x = idx // (S * S)
        y = (idx // S) % S
        z = idx % S

        if mass_mask[x, y, z]:
            # ── MASS TICK: radiate + weak decay ──
            if t % MASS_PERIOD == 0:
                q  = charge[x, y, z]
                cq = col_charge[x, y, z]

                # Radiate: 1 grav + 3 EM + 5 color quanta to random neighbors
                total_rad = GRAV_RAD + EM_RAD + COLOR_RAD
                for step in range(total_rad):
                    rng = _xor(rng)
                    ni = rng % 6
                    nx = (x + NBR[ni, 0]) % S
                    ny = (y + NBR[ni, 1]) % S
                    nz = (z + NBR[ni, 2]) % S
                    if not mass_mask[nx, ny, nz]:
                        if step < GRAV_RAD:
                            v = grav[nx, ny, nz] + 1
                            if v > GRAV_CAP: v = GRAV_CAP
                            grav[nx, ny, nz] = v
                        elif step < GRAV_RAD + EM_RAD:
                            v = em[nx, ny, nz] + q
                            if v > EM_CAP: v = EM_CAP
                            elif v < -EM_CAP: v = -EM_CAP
                            em[nx, ny, nz] = v
                        else:
                            v = color[nx, ny, nz] + cq
                            if v > COLOR_CAP: v = COLOR_CAP
                            elif v < -COLOR_CAP: v = -COLOR_CAP
                            color[nx, ny, nz] = v

                # Reset own fields to sector caps
                grav[x, y, z]  = GRAV_CAP
                em[x, y, z]    = q  * EM_CAP
                color[x, y, z] = cq * COLOR_CAP

                # Attempt weak decay
                rng, did = _try_weak_decay(
                    grav, em, color, mass_mask, charge, col_charge,
                    mpx, mpy, mpz, nm, x, y, z, S, rng)
                if did:
                    stats[8] += 1

        else:
            # ── VACUUM TICK: diffuse each field to a random neighbor ──
            if t % VAC_PERIOD == 0:

                # Gravitational diffusion (unsigned)
                if grav[x, y, z] > 0:
                    rng = _xor(rng)
                    ni = rng % 6
                    nx = (x + NBR[ni, 0]) % S
                    ny = (y + NBR[ni, 1]) % S
                    nz = (z + NBR[ni, 2]) % S
                    grav[x, y, z] -= 1
                    grav[nx, ny, nz] += 1
                    if mass_mask[nx, ny, nz] and grav[nx, ny, nz] > GRAV_CAP:
                        rng, d = _do_fission(
                            grav, em, color, mass_mask, charge, col_charge,
                            mpx, mpy, mpz, nm, nx, ny, nz, x, y, z, S, 0, rng)
                        if d < 0: stats[0] += 1
                        elif d > 0: stats[1] += 1

                # EM diffusion (signed)
                if not mass_mask[x, y, z] and em[x, y, z] != 0:
                    rng = _xor(rng)
                    ni = rng % 6
                    nx = (x + NBR[ni, 0]) % S
                    ny = (y + NBR[ni, 1]) % S
                    nz = (z + NBR[ni, 2]) % S
                    sgn = 1 if em[x, y, z] > 0 else -1

                    # Track annihilation: opposite-sign field meeting
                    val_old = em[nx, ny, nz]
                    if val_old != 0 and ((val_old > 0) != (sgn > 0)):
                        stats[6] += 1   # EM annihilation event

                    em[x, y, z] -= sgn
                    em[nx, ny, nz] += sgn
                    if mass_mask[nx, ny, nz] and abs(em[nx, ny, nz]) > EM_CAP:
                        rng, d = _do_fission(
                            grav, em, color, mass_mask, charge, col_charge,
                            mpx, mpy, mpz, nm, nx, ny, nz, x, y, z, S, 1, rng)
                        if d < 0: stats[2] += 1
                        elif d > 0: stats[3] += 1

                # Color diffusion (signed)
                if not mass_mask[x, y, z] and color[x, y, z] != 0:
                    rng = _xor(rng)
                    ni = rng % 6
                    nx = (x + NBR[ni, 0]) % S
                    ny = (y + NBR[ni, 1]) % S
                    nz = (z + NBR[ni, 2]) % S
                    sgn = 1 if color[x, y, z] > 0 else -1

                    # Track annihilation: opposite-sign color field meeting
                    val_old = color[nx, ny, nz]
                    if val_old != 0 and ((val_old > 0) != (sgn > 0)):
                        stats[7] += 1   # Color annihilation event

                    color[x, y, z] -= sgn
                    color[nx, ny, nz] += sgn
                    if mass_mask[nx, ny, nz] and abs(color[nx, ny, nz]) > COLOR_CAP:
                        rng, d = _do_fission(
                            grav, em, color, mass_mask, charge, col_charge,
                            mpx, mpy, mpz, nm, nx, ny, nz, x, y, z, S, 2, rng)
                        if d < 0: stats[4] += 1
                        elif d > 0: stats[5] += 1

    return rng, stats


# ═══════════════════════════════════════════════════════════════════════════
#   PUBLIC API
# ═══════════════════════════════════════════════════════════════════════════

class Sim:
    """
    RMR Four-Force Simulation.

    A 3D toroidal integer lattice where mass nodes interact through three
    field sectors (gravity, EM, color) governed by a single fission rule:
    E = qΦ.  Weak decay is triggered by adjacency at rate n_adj/137.

    Parameters
    ----------
    S : int
        Lattice size (creates an S×S×S toroidal grid).
    seed : int
        PRNG seed for deterministic reproducibility.
    max_masses : int
        Maximum number of mass nodes (pre-allocated tracking arrays).

    Example
    -------
    >>> sim = Sim(S=35, seed=42)
    >>> sim.add(15, 17, 17, q_em=+1, q_col=0)   # positive EM charge
    >>> sim.add(19, 17, 17, q_em=-1, q_col=0)    # negative EM charge
    >>> for _ in range(3000):
    ...     stats = sim.step()
    >>> print(f"Separation: {sim.mean_separation():.1f}")
    """

    def __init__(self, S, seed=42, max_masses=64):
        self.S = S
        self.grav       = np.zeros((S, S, S), dtype=np.int32)
        self.em         = np.zeros((S, S, S), dtype=np.int32)
        self.color      = np.zeros((S, S, S), dtype=np.int32)
        self.mass_mask  = np.zeros((S, S, S), dtype=np.bool_)
        self.charge     = np.zeros((S, S, S), dtype=np.int8)
        self.col_charge = np.zeros((S, S, S), dtype=np.int8)
        self.mpx = np.full(max_masses, -1, dtype=np.int32)
        self.mpy = np.full(max_masses, -1, dtype=np.int32)
        self.mpz = np.full(max_masses, -1, dtype=np.int32)
        self.nm = 0
        self.rng = np.uint32(seed)
        self.ticks = 0

    def add(self, x, y, z, q_em=0, q_col=0):
        """
        Place a mass node on the lattice.

        Parameters
        ----------
        x, y, z : int
            Lattice coordinates.
        q_em : int
            Electromagnetic charge: -1, 0, or +1.
        q_col : int
            Color charge: -1, 0, or +1.
        """
        self.mass_mask[x, y, z] = True
        self.charge[x, y, z]     = q_em
        self.col_charge[x, y, z] = q_col
        self.grav[x, y, z]       = GRAV_CAP
        self.em[x, y, z]         = q_em  * EM_CAP
        self.color[x, y, z]      = q_col * COLOR_CAP
        self.mpx[self.nm] = x
        self.mpy[self.nm] = y
        self.mpz[self.nm] = z
        self.nm += 1

    def step(self):
        """
        Advance the simulation by one tick.

        Returns
        -------
        stats : ndarray, shape (9,), dtype int32
            [grav_toward, grav_away,
             em_toward, em_away,
             color_toward, color_away,
             em_annihilation, color_annihilation,
             weak_decay]
        """
        self.ticks += 1
        self.rng, stats = _tick(
            self.grav, self.em, self.color, self.mass_mask,
            self.charge, self.col_charge,
            self.mpx, self.mpy, self.mpz, self.nm,
            self.S, self.ticks, self.rng)
        return stats

    def mean_separation(self):
        """Mean pairwise Euclidean distance between all masses on the torus."""
        dists = []
        for i in range(len(self.mpx)):
            if self.mpx[i] < 0:
                continue
            for j in range(i + 1, len(self.mpx)):
                if self.mpx[j] < 0:
                    continue
                d = _eucl(self.mpx[i], self.mpy[i], self.mpz[i],
                          self.mpx[j], self.mpy[j], self.mpz[j], self.S)
                dists.append(d)
        return np.mean(np.array(dists)) if dists else 0.0

    def mass_count(self):
        """Current number of mass nodes (increases with weak decay)."""
        return int(np.sum(self.mass_mask))


# ═══════════════════════════════════════════════════════════════════════════
#   VALIDATION SUITE — reproduces Table 1 from the paper
# ═══════════════════════════════════════════════════════════════════════════
#
#   Seven test conditions × 20 seeds each.  Pre-registered hypotheses:
#     H1: Gravity net_g < 0                        (one-sample t)
#     H2: EM opposite separation < EM like          (Welch t)
#     H3: Strong neutral separation < non-neutral   (Welch t)
#     H4: EM opposite annihilation > EM like        (Welch t)
#     H5: Color neutral annihilation > non-neutral  (Welch t)
#     H6: Weak dense decays > weak isolated         (Welch t)
#

S     = 35
MID   = S // 2
TICKS = 3000
N_SEEDS = 20


def run_one(setup_fn, seed, ticks=TICKS):
    """Run a single simulation and return summary statistics."""
    sim = setup_fn(seed)
    cum = np.zeros(9, dtype=np.int64)
    for _ in range(ticks):
        cum += sim.step()
    return {
        'seed': int(seed),
        'sep': float(sim.mean_separation()),
        'net_g': int(cum[0] - cum[1]),
        'net_e': int(cum[2] - cum[3]),
        'net_c': int(cum[4] - cum[5]),
        'ann_em': int(cum[6]),
        'ann_col': int(cum[7]),
        'weak': int(cum[8]),
        'masses': sim.mass_count(),
    }


# ── Test condition setup functions ──

def mk_gravity(seed):
    """Two neutral, colorless masses separated by 4 lattice units."""
    sim = Sim(S, seed=seed)
    sim.add(MID - 2, MID, MID, q_em=0, q_col=0)
    sim.add(MID + 2, MID, MID, q_em=0, q_col=0)
    return sim

def mk_em_opp(seed):
    """Opposite EM charges (+1, -1), no color."""
    sim = Sim(S, seed=seed)
    sim.add(MID - 2, MID, MID, q_em=+1, q_col=0)
    sim.add(MID + 2, MID, MID, q_em=-1, q_col=0)
    return sim

def mk_em_like(seed):
    """Like EM charges (+1, +1), no color."""
    sim = Sim(S, seed=seed)
    sim.add(MID - 2, MID, MID, q_em=+1, q_col=0)
    sim.add(MID + 2, MID, MID, q_em=+1, q_col=0)
    return sim

def mk_strong_neut(seed):
    """Color-neutral triplet [-1, 0, +1], no EM charge."""
    sim = Sim(S, seed=seed)
    sim.add(MID - 2, MID, MID, q_em=0, q_col=-1)
    sim.add(MID,     MID, MID, q_em=0, q_col= 0)
    sim.add(MID + 2, MID, MID, q_em=0, q_col=+1)
    return sim

def mk_strong_nn(seed):
    """Non-neutral color triplet [+1, +1, +1], no EM charge."""
    sim = Sim(S, seed=seed)
    sim.add(MID - 2, MID, MID, q_em=0, q_col=+1)
    sim.add(MID,     MID, MID, q_em=0, q_col=+1)
    sim.add(MID + 2, MID, MID, q_em=0, q_col=+1)
    return sim

def mk_weak_dense(seed):
    """Dense cluster of 8 masses in a 15³ box (maximizes adjacency)."""
    S4 = 15
    M4 = S4 // 2
    sim = Sim(S4, seed=seed)
    charges = [
        (+1, -1), (-1, +1), (+1, 0), (-1, -1),
        (+1, +1), (-1,  0), (+1, -1), (-1, +1),
    ]
    i = 0
    for dx in range(2):
        for dy in range(2):
            for dz in range(2):
                qe, qc = charges[i]
                i += 1
                sim.add(M4 + dx, M4 + dy, M4 + dz, q_em=qe, q_col=qc)
    return sim

def mk_weak_iso(seed):
    """Isolated pair separated by 10 lattice units (control: no adjacency)."""
    sim = Sim(S, seed=seed)
    sim.add(MID - 5, MID, MID, q_em=+1, q_col=+1)
    sim.add(MID + 5, MID, MID, q_em=-1, q_col=-1)
    return sim


# ═══════════════════════════════════════════════════════════════════════════
#   MAIN: Run full validation suite
# ═══════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    t0 = time.time()
    seeds = [np.uint32(100 + i * 137) for i in range(N_SEEDS)]

    print("JIT warmup...")
    w = Sim(15, seed=1)
    w.add(7, 7, 7, q_em=1, q_col=1)
    w.add(8, 7, 7, q_em=-1, q_col=-1)
    for _ in range(10):
        w.step()
    print("Ready.\n")

    tests = [
        ("Gravity",        mk_gravity),
        ("EM_Opposite",    mk_em_opp),
        ("EM_Like",        mk_em_like),
        ("Strong_Neutral", mk_strong_neut),
        ("Strong_NonNeut", mk_strong_nn),
        ("Weak_Dense",     mk_weak_dense),
        ("Weak_Isolated",  mk_weak_iso),
    ]

    all_res = {}
    for name, fn in tests:
        print(f"Running {name}...", end="", flush=True)
        results = []
        for i, seed in enumerate(seeds):
            results.append(run_one(fn, seed))
            if (i + 1) % 5 == 0:
                print(f" {i + 1}", end="", flush=True)
        all_res[name] = results
        print(f"  ({time.time() - t0:.0f}s)")

    # ── Results table ──
    print(f"\n\n{'=' * 90}")
    print(f"  TABLE 1: v6 Relational Potential E = qΦ  (N={N_SEEDS}, T={TICKS}, S={S})")
    print(f"{'=' * 90}\n")

    print(f"  {'Test':<18} {'Sep':>10} {'Net G':>8} {'Net E':>8} {'Net C':>8} "
          f"{'Ann_E':>7} {'Ann_C':>7} {'Weak':>6} {'M':>4}")
    print(f"  {'-' * 86}")

    sums = {}
    for name, _ in tests:
        res = all_res[name]
        s  = np.array([r['sep'] for r in res])
        ng = np.array([r['net_g'] for r in res])
        ne = np.array([r['net_e'] for r in res])
        nc = np.array([r['net_c'] for r in res])
        ae = np.array([r['ann_em'] for r in res])
        ac = np.array([r['ann_col'] for r in res])
        wk = np.array([r['weak'] for r in res])
        ms = np.array([r['masses'] for r in res])
        sums[name] = {'sep': s, 'ng': ng, 'ne': ne, 'nc': nc,
                       'ae': ae, 'ac': ac, 'wk': wk}
        print(f"  {name:<18} {s.mean():>5.1f}±{s.std():>3.1f} "
              f"{ng.mean():>+7.1f} {ne.mean():>+7.1f} {nc.mean():>+7.1f} "
              f"{ae.mean():>6.0f} {ac.mean():>6.0f} {wk.mean():>5.1f} "
              f"{ms.mean():>3.0f}")

    # ── Hypothesis tests ──
    print(f"\n{'=' * 90}")
    print(f"  HYPOTHESIS TESTS")
    print(f"{'=' * 90}\n")

    def cmp(title, hypothesis, a_name, b_name, metric):
        a = sums[a_name][metric]
        b = sums[b_name][metric]
        t_s, t_p = sp_stats.ttest_ind(a, b, equal_var=False)
        u_s, u_p = sp_stats.mannwhitneyu(a, b, alternative='two-sided')
        ps = np.sqrt((a.std()**2 + b.std()**2) / 2)
        d = abs(a.mean() - b.mean()) / ps if ps > 0 else 0
        sig = ("***" if t_p < 0.001 else "**" if t_p < 0.01
               else "*" if t_p < 0.05 else "ns")
        print(f"  {title}")
        print(f"    H: {hypothesis}")
        print(f"    {a_name}: {a.mean():.2f}±{a.std():.2f}  vs  "
              f"{b_name}: {b.mean():.2f}±{b.std():.2f}")
        print(f"    Welch t={t_s:+.2f}  p={t_p:.6f}  {sig}  "
              f"MW U={u_s:.0f}  p={u_p:.6f}")
        print(f"    Cohen's d = {d:.2f}\n")

    # H1: Gravity
    ng = sums["Gravity"]["ng"]
    t_s, t_p = sp_stats.ttest_1samp(ng, 0)
    sig = ("***" if t_p < 0.001 else "**" if t_p < 0.01
           else "*" if t_p < 0.05 else "ns")
    print(f"  H1: GRAVITY (one-sample, H0: net_grav = 0)")
    print(f"    Net grav: {ng.mean():.2f} ± {ng.std():.2f}")
    print(f"    t={t_s:+.2f}  p={t_p:.6f}  {sig}\n")

    # H2–H3: Separation
    cmp("H2: EM SEPARATION", "Opposite < Like",
        "EM_Opposite", "EM_Like", "sep")
    cmp("H3: STRONG CONFINEMENT", "Neutral < Non-neutral",
        "Strong_Neutral", "Strong_NonNeut", "sep")

    # H4–H5: Annihilation
    cmp("H4: EM ANNIHILATION", "Opposite > 0, Like = 0",
        "EM_Opposite", "EM_Like", "ae")
    cmp("H5: COLOR ANNIHILATION", "Neutral > 0, Non-neutral = 0",
        "Strong_Neutral", "Strong_NonNeut", "ac")

    # H6: Weak decay
    cmp("H6: WEAK DECAY", "Dense > Isolated",
        "Weak_Dense", "Weak_Isolated", "wk")

    # ── Version comparison ──
    eo = sums['EM_Opposite']['sep'].mean()
    el = sums['EM_Like']['sep'].mean()
    sn = sums['Strong_Neutral']['sep'].mean()
    snn = sums['Strong_NonNeut']['sep'].mean()
    print(f"{'=' * 90}")
    print(f"  VERSION HISTORY (v2–v6)")
    print(f"{'=' * 90}")
    print(f"  {'Metric':<30} {'v2':>8} {'v3':>8} {'v4':>8} "
          f"{'v5':>8} {'v6':>8}")
    print(f"  {'─' * 70}")
    print(f"  {'EM Opp-Like Δsep':<30} {'−2.6':>8} {'−0.4':>8} "
          f"{'+0.2':>8} {'+3.1':>8} {eo - el:>+8.1f}")
    print(f"  {'Strong N-NN Δsep':<30} {'−0.9':>8} {'−0.5':>8} "
          f"{'+0.6':>8} {'+0.7':>8} {sn - snn:>+8.1f}")
    print(f"  {'Gravity net_g':<30} {'−16.1':>8} {'−18.2':>8} "
          f"{'−20.9':>8} {'−20.9':>8} "
          f"{sums['Gravity']['ng'].mean():>+8.1f}")
    print(f"  {'EM_Like annihilations':<30} {'0':>8} {'0':>8} "
          f"{'62':>8} {'0':>8} "
          f"{sums['EM_Like']['ae'].mean():>8.0f}")

    print(f"\n  Total runtime: {time.time() - t0:.0f}s "
          f"({(time.time() - t0) / 60:.1f} min)")

JIT warmup...
Ready.

Running Gravity... 5 10 15 20  (35s)
Running EM_Opposite... 5 10 15 20  (63s)
Running EM_Like... 5 10 15 20  (91s)
Running Strong_Neutral... 5 10 15 20  (118s)
Running Strong_NonNeut... 5 10 15 20  (148s)
Running Weak_Dense... 5 10 15 20  (151s)
Running Weak_Isolated... 5 10 15 20  (178s)


  TABLE 1: v6 Relational Potential E = qΦ  (N=20, T=3000, S=35)

  Test                      Sep    Net G    Net E    Net C   Ann_E   Ann_C   Weak    M
  --------------------------------------------------------------------------------------
  Gravity             16.3±4.9   -20.9    +0.0    +0.0      0      0   0.0   2
  EM_Opposite         17.5±2.2    -6.5   -15.4    +0.0   1041      0   0.0   2
  EM_Like             15.4±5.2    -5.9   -13.7    +0.0      0      0   0.0   2
  Strong_Neutral      16.8±2.7   -15.0    +0.4   -28.1     22   1824   0.1   3
  Strong_NonNeut      16.7±2.7    +1.9    -1.6   -38.5     55    111   0.1   3
  Weak_Dense           7.2±0.3   -41.7    -9.2   -